# Aprendizagem SUPERVISIONADA: Classificação

###### Este projeto tem por objetivo desenvolver um algoritmo de Machine Learning para prever a tendência de uma pessoa desenvolcer algum tipo de doença cardiáca com base em alguns fatores clínicos e labratóriais.

# PRÉ PROCESSAMENTO

In [1]:
import numpy as np
import pandas as pd

In [2]:
PATH_ARQUIVO = r'C:\Users\vkrue\OneDrive\Área de Trabalho\GitHub\MachineLearningPython\ModelosClassificacao\Dados\heart_tratado.csv'
df_heart = pd.read_csv(PATH_ARQUIVO, sep=';')

df_heart.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289.0,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180.0,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283.0,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214.0,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195.0,0,Normal,122,N,0.0,Up,0


#### Transformando as variáveis categóricas nominais em variáveis categóricas ordinais

In [3]:
df_heart_2 = pd.DataFrame.copy(df_heart)

In [4]:
df_heart_2['Sex'].replace(
    {
        'M': 0, 
        'F': 1
    }, 
    inplace=True
)

df_heart_2['ChestPainType'].replace(
    {
        'TA': 0, 
        'ATA': 1,
        'NAP': 2,
        'ASY': 3
    }, 
    inplace=True
)

df_heart_2['RestingECG'].replace(
    {
        'Normal': 0, 
        'ST': 1,
        'LVH': 2
    }, 
    inplace=True
)

df_heart_2['ExerciseAngina'].replace(
    {
        'N': 0, 
        'Y': 1
    }, 
    inplace=True
)

df_heart_2['ST_Slope'].replace(
    {
        'Up': 0, 
        'Flat': 1,
        'Down': 2
    },  
    inplace=True
)

In [5]:
df_heart_2.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,0,1,140,289.0,0,0,172,0,0.0,0,0
1,49,1,2,160,180.0,0,0,156,0,1.0,1,1
2,37,0,1,130,283.0,0,1,98,0,0.0,0,0
3,48,1,3,138,214.0,0,0,108,1,1.5,1,1
4,54,0,2,150,195.0,0,0,122,0,0.0,0,0


# Atributos previsores e alvo

In [6]:
previsores = df_heart_2.iloc[:, 0:11].values
previsores

array([[40. ,  0. ,  1. , ...,  0. ,  0. ,  0. ],
       [49. ,  1. ,  2. , ...,  0. ,  1. ,  1. ],
       [37. ,  0. ,  1. , ...,  0. ,  0. ,  0. ],
       ...,
       [57. ,  0. ,  3. , ...,  1. ,  1.2,  1. ],
       [57. ,  1. ,  1. , ...,  0. ,  0. ,  1. ],
       [38. ,  0. ,  2. , ...,  0. ,  0. ,  0. ]])

In [7]:
alvo = df_heart_2.iloc[:, 11].values
alvo

array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,

#### Análise das escalas dos atributos (Escalonamento)

In [8]:
df_heart_2.describe()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
count,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.000000,917.00000,917.000000
mean,53.509269,0.210469,2.251908,132.540894,244.635389,0.233370,0.604144,136.789531,0.404580,0.886696,0.63795,0.552890
std,9.437636,0.407864,0.931502,17.999749,53.347125,0.423206,0.806161,25.467129,0.491078,1.066960,0.60727,0.497466
min,28.000000,0.000000,0.000000,80.000000,85.000000,0.000000,0.000000,60.000000,0.000000,-2.600000,0.00000,0.000000
25%,47.000000,0.000000,2.000000,120.000000,214.000000,0.000000,0.000000,120.000000,0.000000,0.000000,0.00000,0.000000
50%,54.000000,0.000000,3.000000,130.000000,244.635389,0.000000,0.000000,138.000000,0.000000,0.600000,1.00000,1.000000
75%,60.000000,0.000000,3.000000,140.000000,267.000000,0.000000,1.000000,156.000000,1.000000,1.500000,1.00000,1.000000
max,77.000000,1.000000,3.000000,200.000000,603.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.00000,1.000000


#### Padronização (Utiliza a média e o desvio padrão como referência)
#### Normalização (Utiliza os valores máximo e mínimos como referência)

In [9]:
from sklearn.preprocessing import StandardScaler

In [10]:
previsores_esc = StandardScaler().fit_transform(previsores) # Padronização
previsores_esc

array([[-1.43220634, -0.51630861, -1.34470119, ..., -0.82431012,
        -0.83150225, -1.05109458],
       [-0.47805725,  1.9368261 , -0.27058012, ..., -0.82431012,
         0.10625149,  0.59651863],
       [-1.75025603, -0.51630861, -1.34470119, ..., -0.82431012,
        -0.83150225, -1.05109458],
       ...,
       [ 0.37007527, -0.51630861,  0.80354095, ...,  1.21313565,
         0.29380223,  0.59651863],
       [ 0.37007527,  1.9368261 , -1.34470119, ..., -0.82431012,
        -0.83150225,  0.59651863],
       [-1.64423947, -0.51630861, -0.27058012, ..., -0.82431012,
        -0.83150225, -1.05109458]])

In [11]:
previsores_df = pd.DataFrame(previsores_esc)
previsores_df

,0,1,2,3,4,5,6,7,8,9,10
0,-1.432206,-0.516309,-1.344701,0.414627,0.832075,-0.551733,-0.749818,1.383339,-0.824310,-0.831502,-1.051095
1,-0.478057,1.936826,-0.270580,1.526360,-1.212261,-0.551733,-0.749818,0.754736,-0.824310,0.106251,0.596519
2,-1.750256,-0.516309,-1.344701,-0.141240,0.719543,-0.551733,0.491306,-1.523953,-0.824310,-0.831502,-1.051095
3,-0.584074,1.936826,0.803541,0.303453,-0.574578,-0.551733,-0.749818,-1.131075,1.213136,0.575128,0.596519
4,0.052026,-0.516309,-0.270580,0.970493,-0.930931,-0.551733,-0.749818,-0.581047,-0.824310,-0.831502,-1.051095
...,...,...,...,...,...,...,...,...,...,...,...
912,-0.902124,-0.516309,-2.418822,-1.252973,0.363191,-0.551733,-0.749818,-0.188170,-0.824310,0.293802,0.596519
913,1.536257,-0.516309,0.803541,0.636973,-0.968441,1.812470,-0.749818,0.165420,-0.824310,2.356860,0.596519
914,0.370075,-0.516309,0.803541,-0.141240,-2.131275,-0.551733,-0.749818,-0.856061,1.213136,0.293802,0.596519
915,0.370075,1.936826,-1.344701,-0.141240,-0.161960,-0.551733,1.732430,1.461915,-0.824310,-0.831502,0.596519


In [12]:
previsores_df.describe()

,0,1,2,3,4,5,6,7,8,9,10
count,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02,917.000000,9.170000e+02,9.170000e+02,9.170000e+02,9.170000e+02
mean,1.859654e-16,7.748558e-18,1.046055e-16,7.767929e-16,-1.869340e-16,4.649135e-17,0.000000,-5.114048e-16,-1.046055e-16,7.748558e-17,-3.874279e-17
std,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00,1.000546,1.000546e+00,1.000546e+00,1.000546e+00,1.000546e+00
min,-2.704405e+00,-5.163086e-01,-2.418822e+00,-2.920572e+00,-2.994023e+00,-5.517333e-01,-0.749818,-3.016886e+00,-8.243101e-01,-3.269662e+00,-1.051095e+00
25%,-6.900904e-01,-5.163086e-01,-2.705801e-01,-6.971063e-01,-5.745784e-01,-5.517333e-01,-0.749818,-6.596226e-01,-8.243101e-01,-8.315022e-01,-1.051095e+00
50%,5.202558e-02,-5.163086e-01,8.035410e-01,-1.412398e-01,0.000000e+00,-5.517333e-01,-0.749818,4.755658e-02,-8.243101e-01,-2.688500e-01,5.965186e-01
75%,6.881250e-01,-5.163086e-01,8.035410e-01,4.146267e-01,4.194568e-01,-5.517333e-01,0.491306,7.547357e-01,1.213136e+00,5.751284e-01,5.965186e-01
max,2.490407e+00,1.936826e+00,8.035410e-01,3.749826e+00,6.721265e+00,1.812470e+00,1.732430,2.561971e+00,1.213136e+00,4.982571e+00,2.244132e+00


##### LabelEncoder: Transformação de variáveis categóricas em numéricas

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
previsores2 = df_heart.iloc[:, 0:11].values
previsores2[:,1] = LabelEncoder().fit_transform(previsores2[:,1])
previsores2[:,2] = LabelEncoder().fit_transform(previsores2[:,2])
previsores2[:,6] = LabelEncoder().fit_transform(previsores2[:,6])
previsores2[:,8] = LabelEncoder().fit_transform(previsores2[:,8])
previsores2[:,10] = LabelEncoder().fit_transform(previsores2[:,10])
previsores2

array([[40, 1, 1, ..., 0, 0.0, 2],
       [49, 0, 2, ..., 0, 1.0, 1],
       [37, 1, 1, ..., 0, 0.0, 2],
       ...,
       [57, 1, 0, ..., 1, 1.2, 1],
       [57, 0, 1, ..., 0, 0.0, 1],
       [38, 1, 2, ..., 0, 0.0, 2]], dtype=object)

#### Criação de variáceis Dummy - OneHotEncoder
* Utilizado para não influenciar o algoritimo sobre o peso dos dados.

* A = 0
* B = 1
* C = 2
* D = 3

```
A B C D 
1 0 0 0 
0 1 0 0 
0 0 1 0 
0 0 0 1 
```

In [15]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [16]:
previsores3 = ColumnTransformer(
    transformers=[
        ('OneHot', OneHotEncoder(), [1, 2, 6, 8, 10])
    ],
    remainder='passthrough'
).fit_transform(previsores2)

previsores3

(917, 20)

In [17]:
previsores3_esc = StandardScaler().fit_transform(previsores3)
previsores3_esc

array([[-0.51630861,  0.51630861, -1.08542493, ..., -0.55173333,
         1.38333943, -0.83150225],
       [ 1.9368261 , -1.9368261 , -1.08542493, ..., -0.55173333,
         0.75473573,  0.10625149],
       [-0.51630861,  0.51630861, -1.08542493, ..., -0.55173333,
        -1.52395266, -0.83150225],
       ...,
       [-0.51630861,  0.51630861,  0.92129817, ..., -0.55173333,
        -0.85606123,  0.29380223],
       [ 1.9368261 , -1.9368261 , -1.08542493, ..., -0.55173333,
         1.46191489, -0.83150225],
       [-0.51630861,  0.51630861, -1.08542493, ..., -0.55173333,
         1.42262716, -0.83150225]])

# Resumo do PRÉ-PROCESSAMENTO

###### alvo = Variável que se pretende atingir (tem ou não doença cardíaca)
###### previsores = Conjunto de variáveis previsoras com as variáveis transformadas em numéricas manualmente
###### previsores_esc = Conjunto de variáveis previsoras com as variáveis categóricas transformadas em númericas, escalonada.
###### previsores2 = Conjunto de variáveis pevisoras com as variáveis ategóricas transformadas em númericas pelo LabelEncoder.
###### previsores3 = Conjunto de variáveis pevisoras transformadas pelo LabelEncoder e OneHotEncoder, sem escolar.
###### previsores3_esc = Conjunto de variáveis pevisoras transformadas pelo LabelEncoder e OneHotEncoder, escolanada.

# Redução de Dimensionalidade
* Objetivo é selecionar os melhores componentes(atributos) para treinamento do algoritmo, através da análise de correlações entre as variáveis;

##### Análise de Componentes Principais (PCA)
**Seleção de característica:** Seleciona os melhores atributos e utiliza sem transdormações;
**Extração de Características:** Encontra os relacinamentos dos melhores atributos e cria novos atributos. É um algoritmo de aprendizagem não supervisionada. Aplica-se em dados linearmente separáveis.

In [18]:
from sklearn.decomposition import PCA

In [19]:
pca = PCA(n_components=4)

In [20]:
previsores_pca = pca.fit_transform(previsores2)
previsores_pca

array([[  44.01218603,  -36.15137592,   10.64727631,   -9.47126222],
       [ -63.9918455 ,  -13.93849233,   31.69212503,   -5.30052573],
       [  38.53844277,   33.91712222,  -12.48837374,  -21.53860158],
       ...,
       [-113.3467467 ,   23.48595247,   -2.51064362,    1.14935806],
       [  -9.1141306 ,  -35.90083188,    4.81814643,    9.1238555 ],
       [ -70.01342608,  -35.68741686,   12.10984845,  -10.51663588]])

In [21]:
pca.explained_variance_ratio_

array([0.72847929, 0.17183534, 0.08122019, 0.01767209])

In [22]:
pca.explained_variance_ratio_.sum()

0.9992069186607342

##### Kernel PCA
* É um algoritmo de aprendizagem não supervisionada;
* Aplica-se também em dados linearmente não separáveis.

In [23]:
from sklearn.decomposition import KernelPCA

In [24]:
kpca = KernelPCA(n_components=4, kernel='rbf')

In [25]:
previsores_kernel = kpca.fit_transform(previsores2)
previsores_kernel

array([[-0.00249772, -0.00290225, -0.0027372 , -0.00191636],
       [-0.00249877, -0.00290359, -0.00273854, -0.00191743],
       [-0.00249785, -0.00290241, -0.00273736, -0.00191649],
       ...,
       [-0.00249774, -0.00290227, -0.00273723, -0.00191638],
       [-0.00249793, -0.00290252, -0.00273748, -0.00191659],
       [-0.00249772, -0.00290225, -0.0027372 , -0.00191637]])

##### Análise do Discriminante Linear (LDA: Linear Discriminant Analysis)
* Algoritmo de aprendizagem supervisionada, pois utiliza a classe como referência para seleção;
* Aplicado em situações com muitos atributos previsore e também com o atributo alvo com muitas clases;

In [26]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(n_components=1)

In [27]:
previsores_lda = lda.fit_transform(previsores2, alvo)

In [28]:
previsores_lda

array([[-1.47297727],
       [-1.41360735],
       [-0.96705302],
       [ 0.53331037],
       [-1.45783427],
       [-1.78559221],
       [-2.28906835],
       [-1.26844095],
       [ 1.03044735],
       [-1.8033219 ],
       [-2.56103248],
       [ 1.08833702],
       [-1.4298088 ],
       [ 1.02172609],
       [-2.51881925],
       [-0.75582909],
       [-0.27447783],
       [-2.33841952],
       [ 0.50562225],
       [ 0.03155735],
       [-2.89530718],
       [-0.23402029],
       [-2.26833719],
       [ 0.97019565],
       [-1.70998121],
       [-2.0730337 ],
       [ 1.62722174],
       [-0.96486408],
       [-1.52502126],
       [-1.33460621],
       [-0.03948873],
       [-1.44459749],
       [ 0.63936919],
       [ 0.34808635],
       [-2.23945383],
       [-1.48957548],
       [ 2.4746182 ],
       [-2.13691975],
       [-1.54252555],
       [ 0.30741381],
       [-1.8748176 ],
       [-0.26749988],
       [-1.51713745],
       [-1.79186716],
       [ 1.00348547],
       [ 0

In [29]:
lda.explained_variance_ratio_

array([1.])

In [30]:
import pickle

In [31]:
arq1 = open('heart.pkl', 'wb') # Criando arquivo
pickle.dump(alvo, arq1) #Salvando a variável no arquivo

In [32]:
arq1.close()

In [33]:
arq1 = open('heart.pkl', 'rb') # Lendo arquivo

In [34]:
alvo = pickle.load(arq1) #Chamando a variavel

## BASE DE TREINAMENTO E TESTE

* array: Nomes dos atributos previsores e alvo
* test_size: Tamanho em porcentagem de dados de teste
* train_size: Tamanho em porcentagem de dados de treinamento
* random_state: nomeação de um estado aleatório;


In [35]:
from sklearn.model_selection import train_test_split

In [36]:
x_treino, x_teste, y_treino, y_teste = train_test_split(previsores3_esc, alvo, test_size=0.68, random_state=0)

In [37]:
x_treino.shape

(293, 20)

In [38]:
x_teste.shape

(624, 20)

In [39]:
y_treino.shape


(293,)

In [40]:
y_teste.shape


(624,)

# Naive Bayes

In [41]:
from sklearn.naive_bayes import GaussianNB

In [42]:
naive = GaussianNB()
naive.fit(x_treino, y_treino)

GaussianNB()

##### Avaliação do modelo

In [43]:
previsores_naive = naive.predict(x_teste)
previsores_naive

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,

In [44]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [45]:
print(f'Acurácia: {round(accuracy_score(y_teste, previsores_naive) * 100, 2)}')

Acurácia: 86.06


In [46]:
confusion_matrix(y_teste, previsores_naive)

array([[232,  44],
       [ 43, 305]], dtype=int64)

In [47]:
print(classification_report(y_teste, previsores_naive))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84       276
           1       0.87      0.88      0.88       348

    accuracy                           0.86       624
   macro avg       0.86      0.86      0.86       624
weighted avg       0.86      0.86      0.86       624



##### Análise dados de treino

In [48]:
previsores_treino = naive.predict(x_treino)
previsores_treino

array([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1], dtype=int64)

In [49]:
accuracy_score(y_treino, previsores_treino) * 100

86.3481228668942

In [50]:
confusion_matrix(y_treino, previsores_treino)


array([[118,  16],
       [ 24, 135]], dtype=int64)

In [51]:
print(classification_report(y_treino, previsores_treino)
)

              precision    recall  f1-score   support

           0       0.83      0.88      0.86       134
           1       0.89      0.85      0.87       159

    accuracy                           0.86       293
   macro avg       0.86      0.86      0.86       293
weighted avg       0.87      0.86      0.86       293



##### Validação Cruzada

In [52]:
from sklearn.model_selection import KFold, cross_val_score

In [53]:
kfold = KFold(n_splits=30, shuffle=True, random_state=5)

In [54]:
modelo = GaussianNB()
resultado = cross_val_score(modelo, previsores3_esc, alvo, cv=kfold)
print(f'Acurácia Média: {resultado.mean() * 100}')

Acurácia Média: 85.168458781362


#### Naive Bayes = Treino e Teste de 86,06% e  Validação Cruzada de 85,19%. 